# Получение открытых рамок считывания

In [ ]:
!pip install biopython
from Bio import SeqIO

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import numpy as np

In [ ]:
fasta_sequences = SeqIO.parse("fragment15.fasta", "fasta")

In [ ]:
len(fasta_sequences)

In [ ]:
!wget https://ftp.ncbi.nlm.nih.gov/genomes/TOOLS/ORFfinder/linux-i64/ORFfinder.gz
!gzip -d ORFfinder.gz

--2022-12-25 15:59:24--  https://ftp.ncbi.nlm.nih.gov/genomes/TOOLS/ORFfinder/linux-i64/ORFfinder.gz
Resolving ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)... 165.112.9.229, 130.14.250.13, 2607:f220:41f:250::228, ...
Connecting to ftp.ncbi.nlm.nih.gov (ftp.ncbi.nlm.nih.gov)|165.112.9.229|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8386303 (8.0M) [application/x-gzip]
Saving to: ‘ORFfinder.gz’

ORFfinder.gz        100%[===================>]   8.00M  14.0MB/s    in 0.6s    

2022-12-25 15:59:25 (14.0 MB/s) - ‘ORFfinder.gz’ saved [8386303/8386303]



In [ ]:
!chmod +x ORFfinder

In [ ]:
!apt-get install libdw1

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
The following NEW packages will be installed:
  libdw1
0 upgraded, 1 newly installed, 0 to remove and 20 not upgraded.
Need to get 203 kB of archives.
After this operation, 795 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic-updates/main amd64 libdw1 amd64 0.170-0.4ubuntu0.1 [203 kB]
Fetched 203 kB in 1s (372 kB/s)
Selecting previously unselected package libdw1:amd64.
(Reading database ... 124016 files and directories currently installed.)
Preparing to unpack .../libdw1_0.170-0.4ubuntu0.1_amd64.deb ...
Unpacking libdw1:amd64 (0.170-0.4ubuntu0.1) ...
Setting up libdw1:amd64 (0.170-0.4ubuntu0.1) ...
Processing triggers for libc-bin (2.27-3ubuntu1.6) ...


In [ ]:
!./ORFfinder -in fragment15.fasta -out orfs.txt -outfmt 0

In [ ]:
!./ORFfinder -in fragment15.fasta -out orfs.fasta -outfmt 0

In [ ]:
seq = []
with open("orfs.fasta") as orfs:
  res = SeqIO.parse(orfs, "fasta")
with open("orfs.fasta") as orfs:
 for i in SeqIO.parse(orfs, "fasta"):
   seq.append(str(i.seq))

In [ ]:
orfs = {}
with open('orfs.fasta', 'r') as file:
    for i in file.readlines():
        if i.startswith('>') and 'partial' not in i:
            seq = ''
            i = i.strip().split(':')
            k = i[0].split('_')[0].split('|')[-1]
            start = i[1]
            end = i[2].split(' ')[0]
            orfs[k]={'start':int(start),'end':int(end), 'seq':seq}
        else:
            i=i.strip()
            orfs[k]['seq']+=i
orfs_df = pd.DataFrame.from_dict(orfs, orient='index')

Помещаем все рамки в таблицу с метками, показывающими были ли эти рамки уже аннотированы, или они пересекаются с аннотированными

In [ ]:
orfs_df['annotated'] = 0
orfs_df['intersected'] = 0
orfs_df.reset_index(inplace=True)
orfs_df = orfs_df.rename(columns = {'index':'query_id'})
orfs_df.head()

,query_id,start,end,seq,annotated,intersected
0,ORF1,24,143,MKGFRSGLLDRSSPRTSTHPQCHRGSHDGSSHSALKNGS,0,0
1,ORF2,165,611,MPCYLRAYCVTTSKADRAHKTNKRQHPHGSRTKRLGDTRLWGGRRR...,0,0
2,ORF3,648,983,MGTRRSTHRRCRHRNSRRHLRTRYRDGPHHGVDRLAVPRLHPLRTL...,0,0
3,ORF4,1587,1781,MVVLNLIAPEWRMRVPRTEPSTFVASVPSATPMMSRAAASLIGAAE...,0,0
4,ORF5,2127,2216,MRAPMSRNVSTACSHDCFGGALYCGNWAG,0,0


# Анализ данных полученных в BLASTP (база uniprot)

Ищем в базе юнипрот, по таксону бактерии и обрабатываем результаты в датафрейме


In [ ]:
df = pd.read_csv('aligment.csv',  names=['query_id', 'accession_number', 'percent_identity', 'alignment_length', 'mismatches', 'gaps', 'query_start', 'query_end', 's_start', 's_end', 'e-value', 'bit_score', 'percent_positives']) 

In [ ]:
df.query_id = df.query_id.apply(lambda x : x.split('_')[0].split('|')[-1])

In [ ]:
df.sort_values(by=['percent_identity'], ascending=False)

,query_id,accession_number,percent_identity,alignment_length,mismatches,gaps,query_start,query_end,s_start,s_end,e-value,bit_score,percent_positives
482,ORF386,Q6NJY1.1,100.000,106,0,0,1,106,1,106,4.110000e-71,209.0,100.00
582,ORF387,Q6NJY0.1,100.000,150,0,0,1,150,1,150,4.970000e-107,306.0,100.00
1948,ORF997,P61004.1,100.000,15,0,0,16,30,49,63,1.000000e-03,36.2,100.00
1949,ORF998,P61004.1,98.479,263,4,0,2,264,82,344,0.000000e+00,528.0,99.24
821,ORF510,Q6NK10.1,90.244,164,16,0,1,164,6,169,9.610000e-104,297.0,94.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...
93,ORF76,Q9WXX7.1,21.453,289,153,11,71,347,40,266,1.270000e-06,52.8,39.79
282,ORF185,Q5UP47.1,21.362,323,218,10,32,341,22,321,3.720000e-07,56.2,40.87
36,ORF73,Q9RPX0.1,21.324,272,185,5,48,296,10,275,2.370000e-06,52.0,40.07
1436,ORF558,Q7A875.1,20.737,217,160,5,134,350,174,378,2.730000e-04,46.6,41.47


In [ ]:
good_finds = df[(df.percent_identity > 85) & (df['e-value'] < 0.05)].copy()

In [ ]:
good_finds_unique = good_finds.sort_values(by='percent_identity', ascending=False).drop_duplicates(subset=['accession_number'])

In [ ]:
# уникальные Query ID с лучшими Percent Identity
good_finds_unique = good_finds_unique.sort_values(by='percent_identity', ascending=False).drop_duplicates(subset=['query_id'])
good_finds_unique

,query_id,accession_number,percent_identity,alignment_length,mismatches,gaps,query_start,query_end,s_start,s_end,e-value,bit_score,percent_positives
482,ORF386,Q6NJY1.1,100.000,106,0,0,1,106,1,106,4.110000e-71,209.0,100.00
582,ORF387,Q6NJY0.1,100.000,150,0,0,1,150,1,150,4.970000e-107,306.0,100.00
1948,ORF997,P61004.1,100.000,15,0,0,16,30,49,63,1.000000e-03,36.2,100.00
821,ORF510,Q6NK10.1,90.244,164,16,0,1,164,6,169,9.610000e-104,297.0,94.51
1306,ORF550,Q6NJY9.1,87.013,77,10,0,1,77,54,130,8.180000e-45,148.0,93.51


Получаем информацию из genbank

In [ ]:
from Bio import Entrez

record = []

Entrez.email = 'A.N.Other@example.com'

for line in good_finds_unique.values:

  handle = Entrez.efetch(db="protein", id=str(line[1]), rettype="gb")


  record.append(SeqIO.read(handle, "genbank"))

In [ ]:
record

[SeqRecord(seq=Seq('MTQPDMSQILAQAQQMQAKLQEAQREILATTVTGTAGNGLVSIDMQGNGMVSSV...GMF'), id='Q6NJY1.1', name='Y260_CORDI', description='RecName: Full=Nucleoid-associated protein DIP0260', dbxrefs=[]),
 SeqRecord(seq=Seq('MFEGPLQDLIDEFSRLPGVGPKSAQRIAFHLLHVEPADITRLQDALGAIRDGVT...LTL'), id='Q6NJY0.1', name='RECR_CORDI', description='RecName: Full=Recombination protein RecR', dbxrefs=[]),
 SeqRecord(seq=Seq('MIRKDLSQIPTYVPGKRNDHALKLSSNEVTHRPLPSAAQAMAEAAAGANRYPDM...SFA'), id='P61004.1', name='PATR_CORDI', description='RecName: Full=Putative phenylalanine aminotransferase', dbxrefs=[]),
 SeqRecord(seq=Seq('MVIGVLTLQGGFAEHIAILESLGVEHRRVRVPNDLLGLDGLIIPGGESTVMDKL...ASI'), id='Q6NK10.1', name='PDXT_CORDI', description="RecName: Full=Pyridoxal 5'-phosphate synthase subunit PdxT; AltName: Full=Pdx2; AltName: Full=Pyridoxal 5'-phosphate synthase glutaminase subunit", dbxrefs=[]),
 SeqRecord(seq=Seq('MGTFQGAGRYAPSPSGDLHFGNVRTAVLAWLFARHTGRRFLIRVEDVDTQRSSL...KEV'), id='Q6NJY9.1', name='GLUQ_CORDI', descrip

Помещаем в qualifiers

In [ ]:
qualifiers = []
for subrecord in range(len(record)):
  sequence = str(record[subrecord].seq)
  name = record[subrecord].name
  organism  = record[subrecord].annotations['organism']

  protein = record[subrecord].description
  if 'Short' in protein:
    protein = protein[14:protein.index(';')] # название белка
  else:
    protein = protein[14:]

  gene,product = 'None', 'None'
  for i in record[subrecord].features:
    if 'gene' in i.qualifiers.keys(): # если в фичах нет названия гена или белка, будет написано None
      gene = i.qualifiers['gene'][0]
    if 'product' in i.qualifiers.keys():
      product = i.qualifiers['product'][0]
  
  identity = good_finds_unique.iloc[subrecord]['percent_identity'] # идентичность белка
  positives = good_finds_unique.iloc[subrecord]['percent_positives']
  e_value = good_finds_unique.iloc[subrecord]['e-value'] # E-value
  qualifiers.append([sequence,name,organism,identity,positives,e_value,gene,product])

In [ ]:
record[0]

SeqRecord(seq=Seq('MTQPDMSQILAQAQQMQAKLQEAQREILATTVTGTAGNGLVSIDMQGNGMVSSV...GMF'), id='Q6NJY1.1', name='Y260_CORDI', description='RecName: Full=Nucleoid-associated protein DIP0260', dbxrefs=[])

In [ ]:
qualifiers[0]

['MTQPDMSQILAQAQQMQAKLQEAQREILATTVTGTAGNGLVSIDMQGNGMVSSVTIDPKVVDADDVETLQDLLVGAFAEAHEKLGTLAEQKMGPLSQGFDGLGGMF',
 'Y260_CORDI',
 'Corynebacterium diphtheriae NCTC 13129',
 100.0,
 100.0,
 4.11e-71,
 'None',
 'Nucleoid-associated protein DIP0260']

Ставим флаги

In [ ]:
for id in good_finds_unique['query_id'].values:
  orfs_df.loc[orfs_df['query_id'] == id, 'annotated'] = 1

In [ ]:
orfs_df[orfs_df['annotated']==1]

,query_id,start,end,seq,annotated,intersected
385,ORF386,79309,79629,MTQPDMSQILAQAQQMQAKLQEAQREILATTVTGTAGNGLVSIDMQ...,1,0
508,ORF510,47987,48481,MTLQGGFAEQIAILEYLGVEHRRVRVPNDLLGLDGLIIPGGESTVI...,1,0
548,ORF550,70148,70381,MELAARQLEDLHALGMDRDAAPTYQHNNFDRHEHALRKLPHYECYC...,1,0
992,ORF997,41205,41113,MLYAAQAMAEAAPGPNRYPDMGVTELRGAL,1,0


In [ ]:
intersected = []
for value in orfs_df[orfs_df['annotated']==1].values:
  for orf in orfs_df.values:
    if value[1]<orf[1]<value[2] or value[1]<orf[2]<value[2]:
      intersected.append(orf[0])

for id in intersected:
  orfs_df.loc[orfs_df['query_id'] == id, 'intersected'] = 1

In [ ]:
orfs_df[(orfs_df['annotated']==1) | (orfs_df['intersected']==1)]

,query_id,start,end,seq,annotated,intersected
107,ORF108,48330,48410,MRWKESPQTWPLSAPPRLFPAGRACL,0,1
108,ORF109,48411,48539,MRAVATMWWACARARSTPMRSTHDLLVRSDYIRFGWMLCGLK,0,1
165,ORF166,70155,70364,MPPVSWRTCMRWVWIGTPPPPTSTTTLTAMSMRYASCHIMSATAPA...,0,1
304,ORF305,47962,48129,MSADSNRSTDPPGWVCRTDCDPRVFGCGAPSGARAQRPVRSRRPDY...,0,1
305,ORF306,48202,48291,MCWVDLLGHGGKSGEGPADVRLLGCCSTP,0,1
306,ORF307,48388,48492,MRGGRVCDCAQWRPCGGRAPGQDPRLCVPPMIYW,0,1
366,ORF367,69907,70200,MPLLRKQRRASSSCALRRGDLRSLTHKHRYFSVRRALCAKSKVGSS...,0,1
367,ORF368,70282,70776,MRPQGHARGRPRATHHPGPVPRHVPKFNRGATVGASSSIGPAGEGT...,0,1
385,ORF386,79309,79629,MTQPDMSQILAQAQQMQAKLQEAQREILATTVTGTAGNGLVSIDMQ...,1,0
508,ORF510,47987,48481,MTLQGGFAEQIAILEYLGVEHRRVRVPNDLLGLDGLIIPGGESTVI...,1,0


Удаляем аннотированные и пересекающиеся с аннотированными рамки для дальнейшего поиска

In [ ]:
with open("orfs.fasta", "r") as f:
    lines = f.readlines()

annotated_data = pd.DataFrame()
new_lines = []
annotated_seq = []
is_annotated = False
is_sepp = False

for line in lines:
  if line.startswith('>'): # new fasta line
    is_sepp = False # Флаг на разделение полосы в файле вывода
    if is_annotated:  #Если стоит флаг на удаление, то на новой строке он снимается
      is_annotated = False
    for id in orfs_df[(orfs_df['annotated']==1) | (orfs_df['intersected']==1)]["query_id"]: # Поиск среди id
      if id in line: is_annotated = True
  elif not is_annotated:
      new_lines = new_lines[:-1]

  if is_annotated: 
    annotated_seq.extend(line)
    if not is_sepp:
      annotated_seq.extend(' ') # Пустой пробел позволяет не удалять /n
      is_sepp = True
  else: 
    new_lines.extend(line)
    if not is_sepp:
      new_lines.extend(' ')
      is_sepp = True



# создаем файл, по которому будет произведен поиск в других базах данных
with open("orfs_no_uniprot.fasta", "w") as f:
  for line in new_lines:
    f.write(line)

Разделяем файл для поиска в nr

In [ ]:

with open("orfs_no_uniprot.fasta", "r") as f:
    lines = f.readlines()


new_lines = []
new_str = ''

for x in lines:
  if x.startswith('>'):
    new_lines.append(new_str)
    new_str = x
  else:
    new_str += ';' + x

del new_lines[0]

# Высчитываем примерные размеры файлов:
size = int(len(new_lines)/10)

# Создаем 9 файлов с находками, кол-во которых равняется size
str_count = 0
for i in range(9):
  with open("orfs_no_uniprot_" + str(i+1) + ".fasta", "w") as f:
    for line in new_lines[str_count:str_count+size]:
      orf = line.split(';')
      for x in orf:
        f.write(x)
  str_count += size

# Последний файл заполняем оставшимися строчками
with open("orfs_no_uniprot_10.fasta", "w") as f:
  for line in new_lines[str_count:]:
    orf = line.split(';')
    for x in orf:
      f.write(x)

# Анализ данных полученных в BLASTP (база nr)

In [ ]:
!cat nr_*.csv > nr.csv

In [ ]:
df_nr = pd.read_csv('nr.csv',  names=['query_id', 'accession_number', 'percent_identity', 'alignment_length', 'mismatches', 'gaps', 'query_start', 'query_end', 's_start', 's_end', 'e-value', 'bit_score', 'percent_positives']) 

In [ ]:
df_nr.query_id = df_nr.query_id.apply(lambda x : x.split('_')[0].split('|')[-1])

In [ ]:
df_nr.sort_values(by=['percent_identity'], ascending=False)

,query_id,accession_number,percent_identity,alignment_length,mismatches,gaps,query_start,query_end,s_start,s_end,e-value,bit_score,percent_positives
9654,ORF559,WP_014318527.1,100.000,183,0,0,1,183,1,183,6.770000e-128,366.0,100.00
12408,ORF815,WP_014318485.1,100.000,315,0,0,1,315,1,315,0.000000e+00,639.0,100.00
5980,ORF273,WP_014318482.1,100.000,299,0,0,1,299,1,299,0.000000e+00,573.0,100.00
3177,ORF73,WP_003850159.1,100.000,321,0,0,18,338,1,321,0.000000e+00,658.0,100.00
3275,ORF75,WP_003850165.1,100.000,283,0,0,1,283,1,283,0.000000e+00,541.0,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4756,ORF174,CSI40145.1,24.684,158,113,2,66,222,3,155,2.870000e-08,62.4,44.30
10479,ORF614,WP_238823856.1,23.918,439,275,18,43,462,539,937,1.590000e-09,73.2,41.91
10485,ORF614,WP_233486533.1,23.918,439,275,18,43,462,439,837,6.310000e-10,74.3,42.14
10482,ORF614,WP_231131457.1,23.918,439,275,18,43,462,353,751,1.750000e-09,72.8,41.91


In [ ]:
good_finds_nr = df_nr[(df_nr.percent_identity > 85) & (df_nr['e-value'] < 0.05)].copy()

In [ ]:
good_finds_nr

,query_id,accession_number,percent_identity,alignment_length,mismatches,gaps,query_start,query_end,s_start,s_end,e-value,bit_score,percent_positives
106,ORF995,OJH91503.1,87.179,39,5,0,2,40,11,49,5.500000e-14,70.9,94.87
116,ORF995,SBN40138.1,85.185,27,4,0,59,85,50,76,1.870000e-04,49.3,92.59
117,ORF995,SOX56578.1,88.000,25,3,0,62,86,3,27,2.050000e-04,47.0,92.00
120,ORF995,VXC22763.1,91.667,24,2,0,62,85,202,225,1.000000e-03,47.0,91.67
121,ORF995,AGW42470.1,86.957,23,3,0,60,82,86,108,5.000000e-03,43.9,91.30
...,...,...,...,...,...,...,...,...,...,...,...,...,...
13927,ORF962,WP_259605723.1,85.393,89,12,1,1,89,1,88,5.490000e-44,147.0,88.76
13928,ORF962,WP_085734176.1,87.059,85,11,0,17,101,1,85,8.220000e-44,147.0,89.41
13931,ORF962,WP_244652820.1,85.714,84,12,0,18,101,1,84,5.050000e-42,142.0,88.10
13991,ORF962,CAB0580986.1,85.246,61,8,1,1,61,1,60,7.600000e-26,101.0,86.89


In [ ]:
good_finds_nr_unique = good_finds_nr.sort_values(by='percent_identity', ascending=False).drop_duplicates(subset=['accession_number'])
good_finds_nr_unique

,query_id,accession_number,percent_identity,alignment_length,mismatches,gaps,query_start,query_end,s_start,s_end,e-value,bit_score,percent_positives
11906,ORF739,WP_014316235.1,100.000,158,0,0,1,158,10,167,6.130000e-113,327.0,100.00
7547,ORF470,WP_014301219.1,100.000,244,0,0,1,244,1,244,1.670000e-175,492.0,100.00
4768,ORF185,AEX66408.1,100.000,733,0,0,1,733,1,733,0.000000e+00,1478.0,100.00
6698,ORF387,WP_010934156.1,100.000,150,0,0,1,150,1,150,3.640000e-104,306.0,100.00
12408,ORF815,WP_014318485.1,100.000,315,0,0,1,315,1,315,0.000000e+00,639.0,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
12561,ORF820,WP_014366314.1,85.043,234,35,0,16,249,1,234,1.010000e-141,412.0,93.16
4040,ORF132,WP_014318511.1,85.039,127,19,0,1,127,5,131,7.670000e-65,207.0,88.19
8268,ORF534,CAB0845988.1,85.034,147,22,0,15,161,1,147,8.670000e-83,251.0,87.76
8266,ORF534,CAB0993329.1,85.034,147,22,0,15,161,1,147,2.510000e-83,252.0,88.44


In [ ]:
# уникальные Query ID с лучшими Percent Identity
good_finds_nr_unique = good_finds_nr_unique.sort_values(by='percent_identity', ascending=False).drop_duplicates(subset=['query_id'])
good_finds_nr_unique

,query_id,accession_number,percent_identity,alignment_length,mismatches,gaps,query_start,query_end,s_start,s_end,e-value,bit_score,percent_positives
11906,ORF739,WP_014316235.1,100.000,158,0,0,1,158,10,167,6.130000e-113,327.0,100.00
9856,ORF572,OWN76787.1,100.000,243,0,0,7,249,1,243,6.450000e-174,488.0,100.00
11810,ORF735,CAB0488286.1,100.000,72,0,0,9,80,1,72,3.720000e-45,149.0,100.00
226,ORF1005,WP_014302745.1,100.000,259,0,0,1,259,1,259,0.000000e+00,526.0,100.00
326,ORF1013,WP_003850156.1,100.000,305,0,0,1,305,1,305,0.000000e+00,627.0,100.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
5232,ORF223,CAB0875569.1,85.294,68,10,0,4,71,1,68,4.550000e-32,120.0,89.71
1838,ORF14,AEX66297.1,85.222,203,30,0,8,210,3,205,2.550000e-119,347.0,88.18
10640,ORF616,CAB0722214.1,85.135,74,11,0,1,74,1045,1118,2.060000e-21,97.1,90.54
8353,ORF539,WP_021334778.1,85.047,214,32,0,17,230,1,214,7.170000e-114,338.0,88.79


In [ ]:
from Bio import Entrez

record_for_nr = []

Entrez.email = 'A.N.Other@example.com'

for line in good_finds_nr_unique.values:
  # Search for protein records by ID
  handle = Entrez.efetch(db="protein", id=str(line[1]), rettype="gb")

  # Parse the GenBank record
  record_for_nr.append(SeqIO.read(handle, "genbank"))

In [ ]:
qualifiers_nr = []
for subrecord in range(len(record_for_nr)):
  sequence = str(record_for_nr[subrecord].seq)
  name = record_for_nr[subrecord].name
  organism  = record_for_nr[subrecord].annotations['organism']

  protein = record_for_nr[subrecord].description
  if 'Short' in protein:
    protein = protein[14:protein.index(';')] # название белка
  else:
    protein = protein[14:]

  gene,product = 'None', 'None'
  for i in record_for_nr[subrecord].features:
    if 'gene' in i.qualifiers.keys(): # если в фичах нет названия гена или белка, будет написано None
      gene = i.qualifiers['gene'][0]
    if 'product' in i.qualifiers.keys():
      product = i.qualifiers['product'][0]
  
  identity = good_finds_nr_unique.iloc[subrecord]['percent_identity'] # идентичность белка
  positives = good_finds_nr_unique.iloc[subrecord]['percent_positives']
  e_value = good_finds_nr_unique.iloc[subrecord]['e-value'] # E-value
  qualifiers_nr.append([sequence,name,organism,identity,positives,e_value,gene,product])

In [ ]:
record_for_nr[0]

SeqRecord(seq=Seq('MSTAVKVVVMGVSGSGKTTVGTLLAERLGVEYRDGDDLHSQANIEKMASGIPLN...ERS'), id='WP_014316235.1', name='WP_014316235', description='gluconokinase [Corynebacterium diphtheriae]', dbxrefs=[])

In [ ]:
qualifiers_nr[0]

['MSTAVKVVVMGVSGSGKTTVGTLLAERLGVEYRDGDDLHSQANIEKMASGIPLNDDDRWPWLAQVGEWLAAQPAGGVIGCSALKRSYRDVLRRECPEVVFVHVHGAREVLLERMNHREGHFMPASLLDSQFATLQPLENDEVGHVFDVTDSPAQIADAAARWLSERS',
 'WP_014316235',
 'Corynebacterium diphtheriae',
 100.0,
 100.0,
 6.13e-113,
 'None',
 'gluconokinase']

In [ ]:
qualifiers_nr[1]

['VDKLALVTGSTRGLGRVLAQGLIEAGARVVVHGSDAQRAEAAAAELGACGFMACDVRDAAATEAAVSEVIERFGAPDILVNNAGIQRRNPIEVFEDQDWDDIVGVNLSGVFHVTRPVASAMKAAGRGKIVMVGSVQSRLGRATIAPYCATKGGVAMFAQGLAAELSPFNVQVNTLSPGYFDTDMNAALVADEEFSGWVCKRTPAGRWGDPRELVGPLLFLASEASSFVTGQNLAVDGGMTAVV',
 'OWN76787',
 'Corynebacterium diphtheriae bv. mitis',
 100.0,
 100.0,
 6.45e-174,
 'None',
 'gluconate 5-dehydrogenase']

In [ ]:
orfs_df['annotated_nr'] = 0
orfs_df['intersected_nr'] = 0

for id in good_finds_nr_unique['query_id'].values:
  orfs_df.loc[orfs_df['query_id'] == id, 'annotated_nr'] = 1

In [ ]:
intersected = []
for value in orfs_df[orfs_df['annotated_nr']==1].values:
  for orf in orfs_df[(orfs_df['annotated_nr']!=1) & (orfs_df['annotated']!=1) & (orfs_df['intersected']!=1)].values:
    if min(value[1],value[2])<orf[1]<max(value[1],value[2]) or min(value[1],value[2])<orf[2]<max(value[1],value[2]):
      intersected.append(orf[0])

for id in intersected:
  orfs_df.loc[orfs_df['query_id'] == id, 'intersected_nr'] = 1

In [ ]:
orfs_df[(orfs_df['annotated_nr']==1) | (orfs_df['intersected_nr']==1)]

,query_id,start,end,seq,annotated,intersected,annotated_nr,intersected_nr
1,ORF2,165,611,MPCYLRAYCVTTSKADRAHKTNKRQHPHGSRTKRLGDTRLWGGRRR...,0,0,0,1
2,ORF3,648,983,MGTRRSTHRRCRHRNSRRHLRTRYRDGPHHGVDRLAVPRLHPLRTL...,0,0,0,1
4,ORF5,2127,2216,MRAPMSRNVSTACSHDCFGGALYCGNWAG,0,0,0,1
5,ORF6,2334,2438,MLGASRPARPVRWSACACLELLVARPVRSRARSR,0,0,0,1
6,ORF7,2469,2591,MRTPGTVKELSAIAVATTIRCRPARSAAPCSELVNWPCSG,0,0,0,1
...,...,...,...,...,...,...,...,...
1065,ORF1070,4707,4492,MHQTPRKPPDKQRASAAPSAKSTPTGTPLALIDQTTPTPNEPVRPT...,0,0,0,1
1070,ORF1075,2001,1924,MPVGIFLVQARQCLISSGLPARWRG,0,0,0,1
1072,ORF1077,1530,1177,MALSSLRSPIRPLSCAIASDIFTSITVNRGRMLMLTTPPCGLALRS...,0,0,0,1
1073,ORF1078,1104,853,MVSGRAATRRFVPICADATPSILGRRIRRVFVEPLLLCQVLTPHDE...,0,0,0,1


In [ ]:
with open("orfs.fasta", "r") as f:
    lines = f.readlines()

annotated_data = pd.DataFrame()
new_lines = []
annotated_seq = []
is_annotated = False
is_sepp = False

for line in lines:
  if line.startswith('>'): # new fasta line
    is_sepp = False # Флаг на разделение полосы в файле вывода
    if is_annotated:  #Если стоит флаг на удаление, то на новой строке он снимается
      is_annotated = False
    for id in orfs_df[(orfs_df['annotated']==1) | (orfs_df['intersected']==1)]["query_id"]: # Поиск среди id
      if id in line: is_annotated = True
  elif not is_annotated:
      new_lines = new_lines[:-1]

  if is_annotated: 
    annotated_seq.extend(line)
    if not is_sepp:
      annotated_seq.extend(' ') # Пустой пробел позволяет не удалять /n
      is_sepp = True
  else: 
    new_lines.extend(line)
    if not is_sepp:
      new_lines.extend(' ')
      is_sepp = True



# создаем файл, по которому будет произведен поиск в других базах данных
with open("orfs_after_nr.fasta", "w") as f:
  for line in new_lines:
    f.write(line)

# Сколько хороших находок

In [ ]:
len(orfs_df[(orfs_df['annotated']==1) | (orfs_df['annotated_nr']==1)])

95

# Результаты tRNAscan и Aragorn

fragment15_ID_1.trna1 (40001-40085)	Length: 85 bp

Seq:GGAGATGTGGCAGAGCGGCcgAATGCACTGGTCTTGAAAACCAGCGATGGGAAACCATCCCAGGGTTCAAATCCCTGCGTCTCCG

fragment15_ID_1.trna2 (41413-41501)	Length: 89 bp

Seq: GGAGACGTGCCACAGCAGCcgTATGGGGCTCCCTGCTAAGTAGTCGACTTGTTTGCGCAGGTCCGGAGGTTCAACTCCTCTCGTATCCG

fragment15_ID_1.trna3 (41549-41621)	Length: 73 bp

Seq: GCGCCCGTTGCTCAACGGAtAGAGCATCTGACTACGGATCAGAAGGtTGGGGGTTCGAATCCCTCCGGGCGCA

fragment15_ID_1.trna4 (43722-43794)	Length: 73 bp

Seq: GCGCCCGTAGCTAAATGGAtAGAGCATCTGACTACGGAACAGAAGGtTGGCGTTTCAAATCCCTCCGGGCGCA

fragment15_ID_1.trna5 (74938-74853)	Length: 86 bp

Seq: GGAGGATTCGACTAGCGGCctATGTCACACGCCTGGAACGCGTGCGGGTTTCACGACCCTCGTGGGTTCAAATCCCACATCCTCCG

**Aragorn**

tRNA-Ser(tga)
    86 bases, %GC = 58.1
    Sequence [161,246]

tRNA-Ser(gct)
    90 bases, %GC = 60.0
    Sequence [1573,1662]

tRNA-Arg(acg)
    75 bases, %GC = 60.0
    Sequence [1708,1782]

tRNA-Ser(gga)
    87 bases, %GC = 60.9
    Sequence c[35012,35098]

In [ ]:
orfs_df['annotated'] = orfs_df['annotated'] + orfs_df['annotated_nr']
orfs_df = orfs_df.drop(['annotated_nr'],1)

<ipython-input-131-a1de8757bc29>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  orfs_df = orfs_df.drop(['annotated_nr'],1)


In [ ]:
orfs_df['intersected'] = orfs_df['intersected'] + orfs_df['intersected_nr']
orfs_df = orfs_df.drop(['intersected_nr'],1)

<ipython-input-133-47666df5a9f2>:2: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only
  orfs_df = orfs_df.drop(['intersected_nr'],1)


In [ ]:
orfs_df

,query_id,start,end,seq,annotated,intersected
0,ORF1,24,143,MKGFRSGLLDRSSPRTSTHPQCHRGSHDGSSHSALKNGS,0,0
1,ORF2,165,611,MPCYLRAYCVTTSKADRAHKTNKRQHPHGSRTKRLGDTRLWGGRRR...,0,1
2,ORF3,648,983,MGTRRSTHRRCRHRNSRRHLRTRYRDGPHHGVDRLAVPRLHPLRTL...,0,1
3,ORF4,1587,1781,MVVLNLIAPEWRMRVPRTEPSTFVASVPSATPMMSRAAASLIGAAE...,0,0
4,ORF5,2127,2216,MRAPMSRNVSTACSHDCFGGALYCGNWAG,0,1
...,...,...,...,...,...,...
1070,ORF1075,2001,1924,MPVGIFLVQARQCLISSGLPARWRG,0,1
1071,ORF1076,1905,1603,MSALPGKRSRGRCLEMWRGAMARAILWQAVPVLSWGLLVPRVMSDL...,0,0
1072,ORF1077,1530,1177,MALSSLRSPIRPLSCAIASDIFTSITVNRGRMLMLTTPPCGLALRS...,0,1
1073,ORF1078,1104,853,MVSGRAATRRFVPICADATPSILGRRIRRVFVEPLLLCQVLTPHDE...,0,1


In [ ]:
tRNA1 = ['trna1',40001,40085,'GGAGATGTGGCAGAGCGGCcgAATGCACTGGTCTTGAAAACCAGCGATGGGAAACCATCCCAGGGTTCAAATCCCTGCGTCTCCG',0,0]
tRNA2 = ['trna2',41413,41501,'GGAGATGTGGCAGAGCGGCcgAATGCACTGGTCTTGAAAACCAGCGATGGGAAACCATCCCAGGGTTCAAATCCCTGCGTCTCCG',0,0]
tRNA3 = ['trna3',41549,41621,'GGAGATGTGGCAGAGCGGCcgAATGCACTGGTCTTGAAAACCAGCGATGGGAAACCATCCCAGGGTTCAAATCCCTGCGTCTCCG',0,0]
tRNA4 = ['trna4',74938,74853,'GGAGATGTGGCAGAGCGGCcgAATGCACTGGTCTTGAAAACCAGCGATGGGAAACCATCCCAGGGTTCAAATCCCTGCGTCTCCG',0,0]

trna = pd.DataFrame(data = [tRNA1,tRNA2,tRNA3,tRNA4],columns = orfs_df.columns.values)

In [ ]:
orfs_df.append(trna,ignore_index=True).head()

,query_id,start,end,seq,annotated,intersected
0,ORF1,24,143,MKGFRSGLLDRSSPRTSTHPQCHRGSHDGSSHSALKNGS,0,0
1,ORF2,165,611,MPCYLRAYCVTTSKADRAHKTNKRQHPHGSRTKRLGDTRLWGGRRR...,0,1
2,ORF3,648,983,MGTRRSTHRRCRHRNSRRHLRTRYRDGPHHGVDRLAVPRLHPLRTL...,0,1
3,ORF4,1587,1781,MVVLNLIAPEWRMRVPRTEPSTFVASVPSATPMMSRAAASLIGAAE...,0,0
4,ORF5,2127,2216,MRAPMSRNVSTACSHDCFGGALYCGNWAG,0,1


In [ ]:
intersected = []
for value in trna.values:
  for orf in orfs_df[orfs_df['annotated']==1].values:
    if min(value[1],value[2])<orf[1]<max(value[1],value[2]) or min(value[1],value[2])<orf[2]<max(value[1],value[2]):
      intersected.append(value)

for i in intersected:
  print(i[0])

trna1
trna2
trna3
trna4
trna4


In [ ]:
orfs_df['intersected_trna'] = 0

In [ ]:
intersected = []
for value in trna.values:
  for orf in orfs_df[orfs_df['annotated']!=1].values:
    if min(value[1],value[2])<orf[1]<max(value[1],value[2]) or min(value[1],value[2])<orf[2]<max(value[1],value[2]):
      intersected.append(orf[0])

for id in intersected:
  orfs_df.loc[orfs_df['query_id'] == id, 'intersected_trna'] = 1

In [ ]:
orfs_df = orfs_df.drop(['intersected_trna'],1)

In [ ]:
orfs_df = orfs_df.drop(['intersected_trna_annotated'],1)

In [ ]:
import numpy as np

In [ ]:
#orfs_df.loc[np.isin(orfs_df['query_id'],['trna1','trna2','trna3','trna4']),'intersected'] = 1

In [ ]:
#orfs_df[np.isin(orfs_df['query_id'],['trna1','trna2','trna3','trna4'])]

,query_id,start,end,seq,annotated,intersected


# BLASTN для неаннотированных межгенных промежутков

In [ ]:
start = orfs_df[orfs_df['annotated']==1]['start'].values
end = orfs_df[orfs_df['annotated']==1]['end'].values
orfs_df['chromosome'] = 'f15'
chromosome = orfs_df[orfs_df['annotated']==1]['chromosome'].values
bed_columns = ["chromosome", "start", "end"]

In [ ]:
bed = pd.DataFrame(chromosome,columns = ['chromosome'])

In [ ]:
a = np.minimum(start,end)
b = np.maximum(start,end)
start = a
end = b

In [ ]:
bed['start'] = start
bed['end']  = end
bed.to_csv("data.bed", sep="\t", index=False, header=False)
bed.head()

,chromosome,start,end
0,f15,3786,3920
1,f15,4431,5063
2,f15,6159,6848
3,f15,6849,7160
4,f15,7419,7781


In [ ]:
!sudo apt-get install bedtools

Reading package lists... Done
Building dependency tree       
Reading state information... Done
bedtools is already the newest version (2.26.0+dfsg-5).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 20 not upgraded.


Находим промежутки

In [ ]:
!bedtools sort -i data.bed > sorted_data.bed

In [ ]:
!bedtools complement -i sorted_data.bed -g data_genome.bed > intergenic_reg.bed

Error: requested chromosome f15 does not exist in the genome file data_genome.bed. Exiting.


In [ ]:
!bedtools getfasta -fi t.fa -bed ii.bed > intergenic_reg.fasta

index file t.fa.fai not found, generating...


In [ ]:
with open('intergenic_reg.fasta','r') as f:
    f = f.readlines()
file = open('intergenic_reg_filt.fa','w')
for i in f:
    if i.startswith('>'):
        head = i
    else:
        if len(i)>100:
            file.write(head+i)
file.close()

In [ ]:
!wc -l intergenic_reg_filt.fa

92 intergenic_reg_filt.fa


In [ ]:
file = open('unannotated_orfs.bed', 'w')
for i in orfs_df[orfs_df['annotated']==0].values:
        if min(i[1],i[2])==i[1]:
            strand = '+'
        else:
            strand = '-'
        file.write('f15\t{}\t{}\t{}\t*\t{}\n'.format(min(i[1],i[2]), max(i[1],i[2]),i[0],strand))
file.close()

In [ ]:
!bedtools getfasta -s -fi t.fa -bed unannotated_orfs.bed > unannotated_orfs.fasta

print('c (+) цепи')
!grep '(+)' unannotated_orfs.fasta| wc -l
print('c (-) цепи')
!grep '(-)' unannotated_orfs.fasta| wc -l

c (+) цепи
505
c (-) цепи
475


In [ ]:
df_blastn = pd.read_csv('blastn.csv',names=[1,2,3,4,5,6,7,8,9,10,11,12]) 

In [ ]:
df_blastn

,1,2,3,4,5,6,7,8,9,10,11,12
0,f15:68916-68996(+),NR_102568.1,95.455,22,1,0,52,73,1801,1822,0.046,36.5
1,f15:68916-68996(+),NR_076716.1,100.000,19,0,0,52,70,1736,1754,0.046,35.6
2,f15:55492-55620(+),NR_076738.1,89.286,28,3,0,68,95,2468,2441,0.024,38.3


In [ ]:
qualifiers_nr_df = pd.DataFrame(qualifiers_nr)
qualifiers_nr_df['query_id'] = good_finds_nr_unique.query_id.values
qualifiers = pd.DataFrame(qualifiers)
qualifiers['query_id'] = good_finds_unique.query_id.values

In [ ]:
qualifiers = qualifiers.append(qualifiers_nr_df)

In [ ]:
orfs_df = orfs_df.merge(qualifiers,on='query_id',sort='False',how = 'left')

In [ ]:
orfs_df.head()

,query_id,start,end,seq,annotated,intersected,chromosome,0,1,2,3,4,5,6,7
0,ORF1,24,143,MKGFRSGLLDRSSPRTSTHPQCHRGSHDGSSHSALKNGS,0,0,f15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ORF10,3786,3920,MRPGLVHLPRWMGLNQQFAVVQQCREISRSVAGTPLAMHRQQWG,1,0,f15,MGLDQQFAVVQQCREIARSVAGTPLAMHRQQWAYGTMSAYLMSLGL...,OKY22118,Corynebacterium diphtheriae bv. gravis,93.75,100.0,1.270000e-11,None,DNA methylase
2,ORF100,44796,45074,MVCRPMRLRGFRICLGMCCLRGMWCMCGCRRRVCCRRRREFWRLIF...,0,1,f15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ORF1000,39867,39718,MHQHPPCNQAKYSSRSKPPALTVRTCCRPQATIRLPLGNRRSSGLSAPV,0,1,f15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ORF1001,39639,39496,MWRCQRASSCRFLRATASLRRRPWWRLRPRCGRISGCSPAWSPSTCF,0,1,f15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Поиск в InterPro для гипотетических белков

In [ ]:
file = open('unannotated_and_hypotetical.fasta','w')
n,l=0,0
for i in orfs_df.values:
    if i[4] == 0:
        l+=1
        file.write('>{}\n{}\n'.format(i[0],i[3]))
    elif "hypothetical protein" in i[14]:
        n+=1
        file.write('>{}\n{}\n'.format(i[0],i[3]))
print(f'{n} гипотетически предсказаные и {l} неаннотированые')

34 гипотетически предсказаные и 980 неаннотированые


In [ ]:
file = open('hypotetical.fasta','w')
n,l=0,0
for i in orfs_df.values:
    if i[4] == 0:
        l+=1
        #file.write('>{}\n{}\n'.format(i[0],i[3]))
    elif "hypothetical protein" in i[14]:
        n+=1
        file.write('>{}\n{}\n'.format(i[0],i[3]))
print(f'{n} гипотетически предсказаные')

34 гипотетически предсказаные


In [ ]:
file = open('interpro.tsv','r')
interpro = {}
for line in file:
    line = line.split('\t')
    if line[8] != '-':
      if float(line[8]) < 0.01:
        interpro[line[0]]={'gene':'None','product':line[12], 'organism':'None', 'notes':f'InterPro prediction with {line[8]} score'}

In [ ]:
interpro = pd.DataFrame.from_dict(interpro, orient='index')
interpro.reset_index(inplace=True)
interpro = interpro.rename(columns = {'index':'query_id'})

In [ ]:
interpro

,query_id,gene,product,organism,notes
0,ORF125,None,Immunoglobulin-like fold,None,InterPro prediction with 4.6E-7 score
1,ORF995,None,-\n,None,InterPro prediction with 3.7E-13 score
2,ORF294,None,-\n,None,InterPro prediction with 7.8E-11 score


In [ ]:
orfs_df.loc[orfs_df.query_id=='ORF125',7] = 'Immunoglobulin-like fold'

In [ ]:
orfs_df[orfs_df.query_id=='ORF125'][7]

109    Immunoglobulin-like fold
Name: 7, dtype: object

# Создание .gb файла

In [ ]:
final_ = [v for v in orfs_df[orfs_df.annotated==1].values]


final,sort_param = [],[]
for i in final_:
    sort_param.append(i[1])
sort_param = sorted(sort_param)
for i in sort_param:
    for j in final_:
        if i==j[1]:
            final.append(j)

print(f'было аннотировано {len(final)} orf')

было аннотировано 95 orf


In [ ]:
orfs_df

,query_id,start,end,seq,annotated,intersected,chromosome,0,1,2,3,4,5,6,7
0,ORF1,24,143,MKGFRSGLLDRSSPRTSTHPQCHRGSHDGSSHSALKNGS,0,0,f15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,ORF10,3786,3920,MRPGLVHLPRWMGLNQQFAVVQQCREISRSVAGTPLAMHRQQWG,1,0,f15,MGLDQQFAVVQQCREIARSVAGTPLAMHRQQWAYGTMSAYLMSLGL...,OKY22118,Corynebacterium diphtheriae bv. gravis,93.750,100.00,1.270000e-11,None,DNA methylase
2,ORF100,44796,45074,MVCRPMRLRGFRICLGMCCLRGMWCMCGCRRRVCCRRRREFWRLIF...,0,1,f15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,ORF1000,39867,39718,MHQHPPCNQAKYSSRSKPPALTVRTCCRPQATIRLPLGNRRSSGLSAPV,0,1,f15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,ORF1001,39639,39496,MWRCQRASSCRFLRATASLRRRPWWRLRPRCGRISGCSPAWSPSTCF,0,1,f15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,ORF995,41802,41527,MLEYKDTNPRIRQPQVGISAKKSRPAAFVFLSSRRREKGRERTAST...,1,0,f15,MRKARLGGWLGARLVGLVWQAREHEACACHRHRRAWAQPRRPRDCR...,VXC22763,Pseudoclavibacter sp. 8L,91.667,91.67,1.000000e-03,None,conserved hypothetical protein
1071,ORF996,41364,41248,MTQNTRQCPLSTARGATITGMIRKDLSQIPTCVRDGRK,0,0,f15,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1072,ORF997,41205,41113,MLYAAQAMAEAAPGPNRYPDMGVTELRGAL,1,0,f15,MIRKDLSQIPTYVPGKRNDHALKLSSNEVTHRPLPSAAQAMAEAAA...,PATR_CORDI,Corynebacterium diphtheriae NCTC 13129,100.000,100.00,1.000000e-03,pat,Putative phenylalanine aminotransferase
1073,ORF998,41064,40270,MALCQQLVQVTCTPGDAVVYPWRSFEAYPIFVQVVGATPVAVPLTS...,1,0,f15,MIRKDLSQIPTYVPGKRNDHALKLSSNEVTHRPLLSAAQAMAEAAA...,WP_014318486,Corynebacterium diphtheriae,98.859,99.62,0.000000e+00,None,pyridoxal phosphate-dependent aminotransferase


## Определяем организм

In [ ]:
organism = {}
for i in final:
        try:
            organism[i[9]]+=1
        except KeyError:
            organism[i[9]]=1

for k,v in organism.items():
    print(f'{k} - {v}')

Corynebacterium diphtheriae - 62
Corynebacterium diphtheriae bv. gravis - 6
Corynebacterium diphtheriae C7 (beta) - 6
Corynebacterium - 5
Corynebacterium belfantii - 1
Corynebacterium diphtheriae 241 - 1
Corynebacterium diphtheriae INCA 402 - 1
Corynebacterium diphtheriae DSM 43988 - 1
Corynebacterium diphtheriae NCTC 13129 - 4
Corynebacterium sp. 805_CJEI - 1
Pseudoclavibacter sp. 8L - 1
Corynebacterium diphtheriae HC04 - 1
Corynebacterium diphtheriae bv. mitis - 4
Corynebacterium diphtheriae HC03 - 1


Организм - Corynebacterium diphtheriae

## Записываем в файл

In [ ]:
from Bio import SeqIO, Entrez
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.SeqFeature import SeqFeature, FeatureLocation

In [ ]:
output_file = open('f15_answer.gbk', 'w')

# считываем последовательность объекта
sequence_string =''
seq = open('t.fa','r')
for line in seq:
    if not line.startswith('>'):
        sequence_string+=line.strip()
sequence_object = Seq(sequence_string)

# создаем запись
record = SeqRecord(sequence_object,
                    id='alkirdeev',
                    name='fragment15',
                    description='Final project',
                    annotations={"molecule_type": "DNA", "organism":'Corynebacterium diphtheriae'})

# собираем фичи
for i in final:
    if i[0].startswith('trna'):
        type='tRNA'
        qualifiers = {'product':i[13],'note':[i[10],i[12],i[11]]}
    else:
        type='CDS'
        if isinstance([i[10],i[12],i[11]],list):
            notes = f"Организм: {i[9]}, Identities={i[10]}, Positives={i[11]}, E-value={i[12]}"
        else:
            notes = [i[10],i[12],i[11]]
        qualifiers = {'gene':i[13],'product':i[14],'translation':i[7],'note':[i[10],i[12],i[11]]}
    if int(i[1])<int(i[2]):
        strand = 1
        start,end = int(i[1]),int(i[2])
    elif int(i[1])>int(i[2]):
        strand = -1
        start,end = int(i[2]),int(i[1])
    feature = SeqFeature(FeatureLocation(start=start, end=end, strand=strand), type=type, qualifiers=qualifiers)
    record.features.append(feature)

# записываем результаты в файл
SeqIO.write(record, output_file, 'genbank')

1

In [ ]:
file_out = open('rast_conv.bed','w')
record = SeqIO.parse('rast.gbk', "genbank")
rec = next(record) 
for f in rec.features:
  if f.type!='source':
    file_out.write('{}\t{}\t{}\n'.format(f.type,f.location.start,f.location.end))

In [ ]:
file_out = open('f15_answer_conv.bed','w')
record = SeqIO.parse('f15_answer.gbk', "genbank")
rec = next(record) 
for f in rec.features:
  if f.type!='source':
    file_out.write('{}\t{}\t{}\n'.format(f.type,f.location.start,f.location.end))

# Сравниваем с RAST и рассчет IoU

In [ ]:
def IoU_counter(file1,file2):
    data = open(file1,'r')
    for line in data:
        if line.startswith('all'):
            line = line.split('\t')
            if line[1]=='0':
                A=int(line[2])
            elif line[1]=='1':
                AUB=int(line[2])
    data = open(file2,'r')
    for line in data:
        if line.startswith('all'):
            line = line.split('\t')
            if line[1]=='0':
                B=int(line[2])
    return(AUB/(AUB+A+B))

In [ ]:
!bedtools coverage -a f15_answer_conv.bed -b rast_conv.bed -hist > myvsrast.bed
!bedtools coverage -a rast_conv.bed -b f15_answer_conv.bed -hist > rastvsmy.bed
print(f"IoU кастомной аннотации по сравнению с RAST составляет {IoU_counter('myvsrast.bed','rastvsmy.bed')}")

IoU кастомной аннотации по сравнению с RAST составляет 0.5657649796393251
